# B. Configure MintPy Time Series Analysis

*Author: Alex Lewandowski; Alaska Satellite Facility*

The configuration for a MintPy Time Series Analysis is defined in a config file called [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/main/src/mintpy/defaults/smallbaselineApp.cfg).

This file contains all optional parameters with their default settings.

When using MintPy, do not edit this file directly. Instead, you will create a custom config file that contains only the parameters that you wish to update. MintPy will load the custom config, using it to overwrite default settings in `smallbaselineApp.cfg`.

**You must reprocess the time series after making configuration updates**

You can update the configuration without needing to reload the source input data. However, you must rerun the `modify_network` step to register the updated configuration, and then rerun the remaining [smallbaselineApp.py](https://github.com/insarlab/MintPy/blob/main/src/mintpy/smallbaselineApp.py) steps to reprocess the time series with your updates.

If you make updates in this notebook, skip reprocessing the time series, and jump directly to re-plotting or outputting to GeoTiff, you will see results from the previous time series, not from your most recent updates. 

---
## 0. Import Required Software

In [1]:
from ipyfilechooser import FileChooser
import os
from pathlib import Path
from pprint import pprint

import ipywidgets as widgets
import mintpy
import opensarlab_lib as osl
import re
from util import util

---
## 1. Examine Current State of Custom Config File

**Select your custom config file**

- It will be located in your loaded MintPy time series' `MintPy` directory
- It is a text file with your MintPy project name
  - ie., `path/to/MintPy/my_project.txt`

In [2]:
file_chooser_path = util.get_recent_mintpy_config_path()
if file_chooser_path:
    fc = FileChooser(path=file_chooser_path.parent, filename=file_chooser_path.name, select_default=True)
else:
    file_chooser_path = Path.home()
    fc = FileChooser(file_chooser_path, select_default=False)
    
print("Select your custom MintPy config file (MintPy/my_project_name.txt):")
display(fc)

Select your custom MintPy config file (MintPy/my_project_name.txt):


FileChooser(path='/home/jovyan/la2_test/MintPy', filename='smallbaselineApp.cfg', title='', show_hidden=False,…

**Open and display the current contents of the custom config**

In [3]:
if Path(fc.selected) != file_chooser_path:
    util.write_recent_mintpy_config_path(Path(fc.selected))

custom_config_dict = {}

custom_config_path = Path(fc.selected)
with open(custom_config_path, 'r') as f:
    custom_lines = f.readlines()

custom_lines = [l for l in custom_lines if not l.startswith('\n') and not l.strip().startswith('#')]

for l in custom_lines:
    param = l.split(' ')[0]
    info = l.split('= ')[-1].strip()
    custom_config_dict[param] = info

print('Current custom config:\n')
for l in custom_lines:
    print(l)

Current custom config:

mintpy.compute.maxMemory = auto #[float > 0.0], auto for 4, max memory to allocate in GB

mintpy.compute.cluster   = auto #[local / slurm / pbs / lsf / none], auto for none, cluster type

mintpy.compute.numWorker = auto #[int > 1 / all / num%], auto for 4 (local) or 40 (slurm / pbs / lsf), num of workers

mintpy.compute.config    = auto #[none / slurm / pbs / lsf ], auto for none (same as cluster), config name

mintpy.load.processor       = hyp3  #[isce, aria, hyp3, gmtsar, snap, gamma, roipac, nisar], auto for isce

mintpy.load.autoPath        = auto  #[yes / no], auto for no, use pre-defined auto path

mintpy.load.updateMode      = auto  #[yes / no], auto for yes, skip re-loading if HDF5 files are complete

mintpy.load.compression     = auto  #[gzip / lzf / no], auto for no.

mintpy.load.metaFile        = auto  #[path of common metadata file for the stack], i.e.: ./reference/IW1.xml, ./referenceShelve/data.dat

mintpy.load.baselineDir     = auto  #[path of the

---
## 2. Select how you would like to update your time series configuration

In [4]:
config_option_select = osl.select_parameter(
    [
        "Select a few commonly adjusted options",
        "View the full (very long) MintPy config to set any available option"
    ]
)
display(config_option_select)

RadioButtons(layout=Layout(min_width='800px'), options=('Select a few commonly adjusted options', 'View the fu…

---
## 3. Update the Current Configuration 

In [5]:
full_config = 'very long' in config_option_select.value

if not full_config:
    cpu_count = os.cpu_count()
    multithread_option = osl.select_parameter(["Do not use multithreaded processing",
                                         f"Use my {cpu_count} available cores for multithreaded processing"],
                                             description="Select a multithreaded processing option:")
    
    ref_point_option = osl.select_parameter(["Allow MintPy to determine a reference point", 
                                             "Define a reference point"],
                                           description="Select a refernce point option:")

    ref_date_option = osl.select_parameter(["Reference time-series to earliest date in stack",
                                            "Allow MintPy to determine reference date"],
                                          description="Select a reference date option:")

    tropo_correct_option = osl.select_parameter(["Do not perform tropospheric correction",
                                            "Perform tropospheric correction"],
                                          description="Select a tropospheric correction option:")

    display(multithread_option)
    display(ref_point_option)
    display(ref_date_option)
    display(tropo_correct_option)

RadioButtons(description='Select a multithreaded processing option:', layout=Layout(min_width='800px'), option…

RadioButtons(description='Select a refernce point option:', layout=Layout(min_width='800px'), options=('Allow …

RadioButtons(description='Select a reference date option:', layout=Layout(min_width='800px'), options=('Refere…

RadioButtons(description='Select a tropospheric correction option:', layout=Layout(min_width='800px'), options…

In [6]:
if not full_config:
    updated_config = []
    multithread = 'Use' in multithread_option.value
    mintpy_ref_point = 'Allow' in ref_point_option.value
    mintpy_ref_date = 'Allow' in ref_date_option.value
    tropo_correct = 'Do not' not in tropo_correct_option.value

    for l in custom_lines:
        if 'compute' not in l \
        and 'reference' not in l \
        and 'troposphericDelay' not in l:
            updated_config.append(l)

    if multithread:
        updated_config.append('mintpy.compute.cluster = local')
        updated_config.append(f'mintpy.compute.numWorker = {cpu_count}')
        
    if not mintpy_ref_date:
        updated_config.append(f'mintpy.reference.date = no')
        
    if tropo_correct:
        updated_config.append('mintpy.troposphericDelay.method = pyaps')
    else:
        updated_config.append('mintpy.troposphericDelay.method = no')

    if not mintpy_ref_point:
        is_float = False
        while not is_float:
            try:
                lat = float(input("Enter reference latitude"))
                lon = float(input("Enter reference longitude"))
                is_float = True
            except ValueError:
                print("Latitude and Longitude must be convertable to float")
                continue
            updated_config.append(f'mintpy.reference.lalo = {lat},{lon}')

    updated_config = '\n'.join(updated_config)
    

Enter reference latitude 3758884.1
Enter reference longitude 402864.76


In [7]:
if not full_config:
    print('Current updated custom config:\n')
    pprint(updated_config)
    print('Proceed to the end of the notebook to save the updated config.')

Current updated custom config:

('mintpy.load.processor       = hyp3  #[isce, aria, hyp3, gmtsar, snap, gamma, '
 'roipac, nisar], auto for isce\n'
 '\n'
 'mintpy.load.autoPath        = auto  #[yes / no], auto for no, use '
 'pre-defined auto path\n'
 '\n'
 'mintpy.load.updateMode      = auto  #[yes / no], auto for yes, skip '
 're-loading if HDF5 files are complete\n'
 '\n'
 'mintpy.load.compression     = auto  #[gzip / lzf / no], auto for no.\n'
 '\n'
 'mintpy.load.baselineDir     = auto  #[path of the baseline dir], i.e.: '
 './baselines\n'
 '\n'
 'mintpy.load.unwFile         = /home/jovyan/la2_test/*/*_unw_phase*.tif  '
 '#[path pattern of unwrapped interferogram files]\n'
 '\n'
 'mintpy.load.corFile         = /home/jovyan/la2_test/*/*_corr*.tif  #[path '
 'pattern of spatial coherence       files]\n'
 '\n'
 'mintpy.load.connCompFile    = auto  #[path pattern of connected '
 'components    files], optional but recommended\n'
 '\n'
 'mintpy.load.intFile         = auto  #[path patter

**Read in the default config**

In [8]:
if full_config:
    default_config_path = Path(mintpy.__file__).parent / "defaults/smallbaselineApp.cfg"
    with open(default_config_path, 'r') as f:
        lines = f.readlines()
        
    lines = [l for l in lines if not l.startswith('\n')]

**Complete an interactive config form**

***Only complete the fields you wish to change***

- The form contains settings from your existing custom config and parameter hints for any unused attributes
- Uncompleted fields will be set to 'auto'

**Some Commonly Updated Options:**

- Set a reference point
  -  `mintpy.reference.lalo`
  -  `mintpy.reference.yx`
- Use the time series start date as the reference date
  - `mintpy.reference.date = no`
- Use multihreaded processing for inversion and DEM error correction
  - `mintpy.compute.cluster = local`
  - `mintpy.compute.numWorker`
- Subset
  - `mintpy.subset.lalo`
  - `mintpy.subset.yx`
- Use or turn off tropospheric correction
  - `mintpy.troposphericDelay.method = pyaps`
  - `mintpy.troposphericDelay.method = no`

In [9]:
if full_config:
    param_dict = {}
    
    layout = widgets.Layout(width='initial', height='40px') #set width and height
    
    for l in lines:
        if l.startswith('#'):
            print(l)
        else:
            param = l.split(' ')[0]
            if param in custom_config_dict.keys():
                info = custom_config_dict[param]
            else:
                info = l.split('= auto ')[-1].strip()[1:]  
            param_dict[param] =  widgets.Text(
                placeholder=info,
                description=f'{param}:',
                disabled=False,
                align_items='stretch', 
                layout = layout,
                style= {'description_width': 'initial'},
            )
            display(param_dict[param])

**Create updated list of custom configuration settings**

- Check your settings before overwriting your config file
- Any settings not included in your custom config will be treated as `auto` by MintPy

In [10]:
if full_config:
    updated_config = ""
    
    for k, v in param_dict.items():
        if len(v.value) > 0 and v.value != v.placeholder:
            updated_config += f'{k} = {v.value}\n'
        elif k in custom_config_dict.keys() and v.placeholder == custom_config_dict[k]:
            updated_config += f'{k} = {v.placeholder}\n'
    print('Updated custom config settings:\n')
    pprint(updated_config)

**Overwrite your custom config with updated settings**

- If you are happy with the output from the previous cell, overwrite your config file

In [11]:
custom_config_path.write_text(updated_config)

9845

**It is now time to run the `4_MintPy_Time_Series.ipynb` (Perform MintPy Time Series Analysis) Notebook**

If you skip rerunning the time series and create plots or Geotiff output now, the changes you just made will not be applied. 